In [6]:
import numpy as np
import scipy as sp
import pandas as pd
import pysindy as ps
import matplotlib.pyplot as plt
import warnings

from utilities import *
from bootstrapping import *
from optimizers import *
from weak_forms import *

In [7]:
def run_search(Theta, X_dot, var, 
               n_bootstraps = 100, n_features_to_drop = 1, n_max_features = 5,
               feature_names_list = [" "], print_hierarchy = 0):
    """
    Performs a searcg of the ALASSO solution (regularization) path, indentifies supports and fits them with OLS, returning the optimal model
    Can bootstrap data (n_bootstraps) and features (n_features_to_drop) randomly in the feature selection (support finding) process. 
    Only keeps supports with a number of features below n_max_features 
    IN: Theta [n_points, n_features], X_dot [n_points], supports [n_supports, n_features]

    """
    coefs = np.zeros((Theta.shape[1], 100, n_bootstraps))
    sup = np.zeros_like(coefs[:,:,0])
    supports = []
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        for i in range(n_bootstraps):
            if n_bootstraps != 1:
                Theta_new, X_dot_new, inds = bootstrapping(Theta, X_dot, n_features_to_drop = n_features_to_drop)
            else:
                Theta_new, X_dot_new, inds = Theta, X_dot, np.arange(Theta.shape[1])
            alphas, coefs[inds, :, i] = ALASSO_path(Theta_new, X_dot_new[:,var].reshape(-1,1),)
            supports += identify_unique_supports(coefs[:,:,i], n_max_features = n_max_features)
            sup += np.abs(coefs[:,:,i]) > 0
        
    supports = remove_duplicates(supports)
    coef_list, score, n_terms = fit_supports(Theta, X_dot[:,var], supports)
    opt_coefs, index_min = find_optimal_support(coef_list, score, n_terms)
    print_hierarchy_f(print_hierarchy, coef_list, n_terms, score, feature_names_list)
    return opt_coefs, score[index_min]

In [ ]:
X, t = generate_data(10, 10/0.02, "lorenz")
X = add_noise(X, 0.05)
X_dot = ps.SmoothedFiniteDifference()(X, t)

Theta = np.vstack((np.ones_like(X[:,0]),
                X[:,0], X[:,1], X[:,2], 
                X[:,0]*X[:,0], X[:,1]*X[:,1], X[:,1]*X[:,2],
                X[:,0]*X[:,1], X[:,0]*X[:,2], X[:,1]*X[:,2])).T
feature_name_list = ["1", "x", "y", "z", "x^2", "y^2", "z^2", "x y", "x z", "y z"]

opt_coefs = np.zeros((X.shape[1], Theta.shape[1]))

for var in range(X.shape[1]):
    opt_coefs[var], score = run_search(Theta, X_dot, var,
                            n_bootstraps=1, n_features_to_drop=0,
                            feature_names_list=feature_name_list, print_hierarchy=0)
    
